In [1]:
import pandas as pd
import numpy as np
import random
import time
import project
import founder
import requests
from bs4 import BeautifulSoup
from project import get_project, get_rewards
from founder import get_profile, get_bio
from featurizer1 import featurize
from pymongo import MongoClient
import urllib2

import multiprocessing
import sys
import threading
from timeit import Timer



In [2]:
# Initialize MongoDB database and collection
client = MongoClient()
db = client['ksdb']
collection = db['ksdata']

# Load array of project_id,founder_id, project_url, founder_url, rewards_url

df = pd.read_csv('id_url_list.csv', dtype=object, header=None)
id_url_list = df.values



In [4]:
# function to scrape Kickstarter website based on the provided id_url_list.csv
def pool_extract(gen):
    failed = 0
    skipped = 0
    progress = 0
    
    project_id,founder_id, project_url, founder_url, rewards_url = gen

    collection_check = set(db.ksdata.distinct('project_id', {}))
    if int(project_id) in collection_check:
            print "Already scraped"
            skipped += 1
    else:
        try:
            project_soup, project_url, status1 = get_project(project_url)

            founder_soup, founder_url, status2 = get_profile(founder_url)

            rewards_soup, rewards_url, status3 = get_rewards(rewards_url)

            if (status1 & status2 & status3) == 200:
                featurize(project_id, founder_id, project_url, founder_url, rewards_url, project_soup, founder_soup, rewards_soup, collection)

                progress += 1

                wait = True

        except requests.ConnectionError:
            failed +=1
    
    print '\n'

    print 'Scraped: {}'.format(progress)

    print 'Skipped: {}'.format(skipped)

    print 'Failed: {}'.format(failed)



In [ ]:
pool = multiprocessing.Pool(3)
pool.map(pool_extract, id_url_list)
pool.close()
pool.join()